In [32]:
using CSV, DataFrames, Random, Gurobi

In [33]:
# Read in the data
df = CSV.read("../../data/preprocessed/stock_prices_train.csv",DataFrame)
# remove missing 
df = df[completecases(df),:]
# drop RowID, Date, SecuritiesCode from df
y = df[:, :Target]
#X = df[:, Not([1,2,3,12])] why removing 3 and 12?
X = df[:, Not([1,2])]
# split into train/test
train_size = Int(floor(0.8*size(X,1)))
X_train = X[1:train_size,:]
X_test = X[train_size+1:end,:]
y_train = y[1:train_size]
y_test = y[train_size+1:end]

466507-element Vector{Float64}:
  0.0076267384477344
 -0.0160284951024042
  0.0167420814479637
 -0.0111259457053849
  0.0022502250225022
  0.0035922766052987
  0.0062639821029082
 -0.0022232103156958
  0.0164884135472371
  0.0258658483121438
  0.0192307692307691
 -0.0054507337526205
 -0.0126475548060708
  ⋮
  0.0
 -0.0055865921787709
 -0.0126404494382021
  0.0042674253200569
 -0.0254957507082153
 -0.0290697674418605
 -0.0014970059880239
  0.0269865067466266
 -0.0014598540145985
  0.0175438596491228
  0.014367816091954
  0.0184135977337109

In [34]:
first(df, 10)

Row,Date,SecuritiesCode,Open,High,Low,Volume,Target,AdjustedClose,return_20days,volatility_20days,return_40days,volatility_40days,return_60days,volatility_60days,Drawdown
,Date,Int64,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2017-01-04,1301,2734.0,2755.0,2730.0,31400,0.00073046,2742.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-01-05,1301,2743.0,2747.0,2735.0,17900,0.00291971,2738.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-01-06,1301,2734.0,2744.0,2720.0,19900,-0.0010917,2740.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00401141
4,2017-01-10,1301,2745.0,2754.0,2735.0,24200,-0.00510018,2748.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00801989
5,2017-01-11,1301,2748.0,2752.0,2737.0,9300,-0.0032955,2745.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0062152
6,2017-01-12,1301,2745.0,2747.0,2703.0,28700,-0.00661278,2731.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00953249
7,2017-01-13,1301,2707.0,2730.0,2707.0,19400,-0.0066568,2722.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00957651
8,2017-01-16,1301,2725.0,2725.0,2696.0,20100,0.00297841,2704.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2017-01-17,1301,2702.0,2704.0,2682.0,18400,0.00185598,2686.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00112243


In [35]:
first(X_train, 10)

Row,Open,High,Low,Volume,Target,AdjustedClose,return_20days,volatility_20days,return_40days,volatility_40days,return_60days,volatility_60days,Drawdown
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2734.0,2755.0,2730.0,31400,0.00073046,2742.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2743.0,2747.0,2735.0,17900,0.00291971,2738.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2734.0,2744.0,2720.0,19900,-0.0010917,2740.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00401141
4,2745.0,2754.0,2735.0,24200,-0.00510018,2748.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00801989
5,2748.0,2752.0,2737.0,9300,-0.0032955,2745.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0062152
6,2745.0,2747.0,2703.0,28700,-0.00661278,2731.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00953249
7,2707.0,2730.0,2707.0,19400,-0.0066568,2722.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00957651
8,2725.0,2725.0,2696.0,20100,0.00297841,2704.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2702.0,2704.0,2682.0,18400,0.00185598,2686.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00112243


In [36]:
df_test = CSV.read("../../data/preprocessed/stock_prices_supplemental.csv",DataFrame)
first(df_test, 10)

Row,Date,SecuritiesCode,Open,High,Low,Volume,Target,AdjustedClose,return_20days,volatility_20days,return_40days,volatility_40days,return_60days,volatility_60days,Drawdown
,Date,Int64,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2021-12-06,1301,2982.0,2982.0,2965.0,8900,-0.00326264,2971.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2021-12-07,1301,2998.0,3065.0,2990.0,19100,0.00981997,3065.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2021-12-08,1301,3080.0,3080.0,3035.0,11600,0.00648298,3055.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00333699
4,2021-12-09,1301,3050.0,3085.0,3025.0,11700,-0.00644122,3085.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0162612
5,2021-12-10,1301,3100.0,3105.0,3050.0,14700,-0.00810373,3105.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0179237
6,2021-12-13,1301,3110.0,3110.0,3075.0,12100,0.00980392,3085.0,0.0,0.0,0.0,0.0,0.0,0.0,1.60457e-5
7,2021-12-14,1301,3085.0,3085.0,3055.0,10200,0.0113269,3060.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2021-12-15,1301,3060.0,3095.0,3060.0,8500,-0.0096,3090.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0209269
9,2021-12-16,1301,3100.0,3135.0,3085.0,16900,-0.0242326,3125.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0355595


In [37]:
df = vcat(df,df_test)
first(df, 10)

Row,Date,SecuritiesCode,Open,High,Low,Volume,Target,AdjustedClose,return_20days,volatility_20days,return_40days,volatility_40days,return_60days,volatility_60days,Drawdown
,Date,Int64,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2017-01-04,1301,2734.0,2755.0,2730.0,31400,0.00073046,2742.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-01-05,1301,2743.0,2747.0,2735.0,17900,0.00291971,2738.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-01-06,1301,2734.0,2744.0,2720.0,19900,-0.0010917,2740.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00401141
4,2017-01-10,1301,2745.0,2754.0,2735.0,24200,-0.00510018,2748.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00801989
5,2017-01-11,1301,2748.0,2752.0,2737.0,9300,-0.0032955,2745.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0062152
6,2017-01-12,1301,2745.0,2747.0,2703.0,28700,-0.00661278,2731.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00953249
7,2017-01-13,1301,2707.0,2730.0,2707.0,19400,-0.0066568,2722.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00957651
8,2017-01-16,1301,2725.0,2725.0,2696.0,20100,0.00297841,2704.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2017-01-17,1301,2702.0,2704.0,2682.0,18400,0.00185598,2686.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00112243


In [5]:
df[!,:Drawdown] = zeros(size(df,1))
security_code = df[1204,:SecuritiesCode]
df_prev = df[1:1203,:]
idx = df_prev[!, :SecuritiesCode] .== security_code
print("a")
df[1204,:Drawdown] = maximum(df_prev[idx, :Target]) - df[1204,:Target]

a

0.0349326393141456

In [ ]:
df[!,:Drawdown] = zeros(size(df,1))

for i in 2:size(df,1)
    security_code = df[i,:SecuritiesCode]
    df_prev = df[1:i-1,:]
    idx = df_prev[!, :SecuritiesCode] .== security_code
    df[i,:Drawdown] = maximum(df_prev[idx, :Target], init=0) - df[i,:Target]
end
first(df, 10)

In [87]:
using Dates, Gurobi, JuMP

function prescription_baseline(df, C, D)
    Random.seed!(42)
    m = Model(Gurobi.Optimizer)
    set_optimizer_attribute(m, "OutputFlag", 0)

    #extract the part of the dataset that correspond to day 2022/06/23, which is the day before the test day
    idx = df[!,:Date] .== Date(2022,6,23)
    df_2022_06_23 = df[idx,:]
    
    # extract the 200 performing stocks based on the value of Target for the date "2022-06-23"
    df_2022_06_23_top = sort(df_2022_06_23, :Target, rev=true)[1:200,:]
    # same for the 200 worst performing stocks
    df_2022_06_23_bottom = sort(df_2022_06_23, :Target, rev=false)[1:200,:]

    #we select the top 200 stocks to buy and the bottom 200 stocks to shorts
    @variable(m, w[1:400]>=0);
    @objective(m, Max, sum(w[i] * df_2022_06_23_top[!,"Target"][i] for i in 1:200) + sum(w[i] * df_2022_06_23_bottom[!,"Target"][i] for i in 1:200))
    
    #add the constraint that the max of drawdowns on the day 2022/06/23 weighted by w is less that C
    @constraint(m, (sum(w[i] * df_2022_06_23_top[!,"Drawdown"][i] for i in 1:200) + sum(w[i] * df_2022_06_23_bottom[!,"Drawdown"][i] for i in 1:200))/600 <= C)
    
    #sum of w should be equal to D
    @constraint(m, sum(w) == D)
    optimize!(m)
    #return the value of w
    return value.(w)

end

prescription_baseline (generic function with 1 method)

In [88]:
w_opt_baseline = prescription_baseline(df, 0.01, 600)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18


400-element Vector{Float64}:
  0.0
  0.0
 29.016112399932855
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  ⋮
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [45]:
idx = df[!,:Date] .== Date(2022,6,24)
df_2022_06_24 = df[idx,:]
    
# extract the 200 performing stocks based on the value of Target for the date "2022-06-23"
df_2022_06_24_top = sort(df_2022_06_24, :Target, rev=true)[1:200,:]
# same for the 200 worst performing stocks
df_2022_06_24_bottom = sort(df_2022_06_24, :Target, rev=false)[1:200,:]

Row,Date,SecuritiesCode,Open,High,Low,Volume,Target,AdjustedClose,return_20days,volatility_20days,return_40days,volatility_40days,return_60days,volatility_60days,Drawdown
,Date,Int64,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2022-06-24,4772,185.0,221.0,181.0,6564100,-0.068,203.0,157.6,0.0565635,149.775,0.0498302,149.717,0.0431327,0.299527
2,2022-06-24,3660,232.0,239.0,229.0,1076100,-0.0472103,235.0,229.85,0.045342,203.025,0.0442826,194.3,0.0397477,0.176675
3,2022-06-24,3031,1840.0,1899.0,1816.0,1153500,-0.0436836,1826.0,1533.25,0.0506802,1487.47,0.0425432,1408.03,0.0375405,0.160047
4,2022-06-24,1407,4155.0,4460.0,3805.0,1700800,-0.0431566,4390.0,4272.0,0.0362882,4516.0,0.037786,4693.58,0.0370481,0.232346
5,2022-06-24,4058,1323.0,1388.0,1317.0,19400,-0.0393701,1387.0,1367.75,0.0228054,1412.65,0.0299982,1476.85,0.0287428,0.154196
6,2022-06-24,6094,1842.0,1887.0,1781.0,83800,-0.0387513,1876.0,1757.75,0.0395632,1809.33,0.0426152,1877.73,0.039437,0.156398
7,2022-06-24,4449,1138.0,1249.0,1138.0,2151600,-0.0384924,1233.0,1104.95,0.0573985,1079.35,0.0669403,1130.9,0.0615741,0.220974
8,2022-06-24,7630,4945.0,4975.0,4930.0,27600,-0.0365112,4975.0,4897.0,0.00878699,4886.5,0.0091148,4816.75,0.00886153,0.0606075
9,2022-06-24,4054,1050.0,1073.0,1038.0,56600,-0.0362745,1042.0,1180.25,0.0270262,1202.78,0.0501727,1199.32,0.0483266,0.208072


In [89]:
sum(w_opt_baseline[i] * df_2022_06_24_top[!,"Target"][i] for i in 1:200) + sum(w_opt_baseline[i] * df_2022_06_24_bottom[!,"Target"][i] for i in 1:200)

2.115695049694926

In [91]:
function oracle_policy(df_2022_06_24_top, df_2022_06_24_bottom, C, D)
    Random.seed!(42)
    m = Model(Gurobi.Optimizer)
    set_optimizer_attribute(m, "OutputFlag", 0)


    #we select the top 200 stocks to buy and the bottom 200 stocks to shorts
    @variable(m, w[1:400]>=0);
    @objective(m, Max, sum(w[i] * df_2022_06_24_top[!,"Target"][i] for i in 1:200) + sum(w[i] * df_2022_06_24_bottom[!,"Target"][i] for i in 1:200))
    
    #add the constraint that the max of drawdowns on the day 2022/06/23 weighted by w is less that C
    @constraint(m, (sum(w[i] * df_2022_06_24_top[!,"Drawdown"][i] for i in 1:200) + sum(w[i] * df_2022_06_24_bottom[!,"Drawdown"][i] for i in 1:200))/600 <= C)
    
    #sum of w should be equal to D
    @constraint(m, sum(w) == D)
    optimize!(m)
    #return the value of w
    return value.(w)

end

oracle_policy (generic function with 1 method)

In [92]:
w_opt_oracle = oracle_policy(df_2022_06_24_top, df_2022_06_24_bottom, 0.01, 600)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18


400-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [93]:
sum(w_opt_oracle[i] * df_2022_06_24_top[!,"Target"][i] for i in 1:200) + sum(w_opt_oracle[i] * df_2022_06_24_bottom[!,"Target"][i] for i in 1:200)

3.138319003050813

In [77]:
using NearestNeighbors, Statistics
df_train = df[df[:, :Date] .!= Date(2022,6,24),:]
X_train = df_train[:, Not([1,2,7,15])]
y_train = df_train[:, :Target]
X_test_top = df_2022_06_24_top[:, Not([1,2,7,15])]
y_test_top = df_2022_06_24_top[:, :Target]
X_test_bottom = df_2022_06_24_bottom[:, Not([1,2,7,15])]
y_test_bottom = df_2022_06_24_bottom[:, :Target]

num_neighbors = 25
kdtree = KDTree(Matrix(X_train)'; leafsize = 10)
indices_top, dists_top = knn(kdtree, Matrix(X_test_top)', num_neighbors)
indices_bottom, dists_bottom = knn(kdtree, Matrix(X_test_bottom)', num_neighbors)
println(indices_top[1])
#z = round(sum(y_train[i] for i in indices) / num_neighbors)
#MAE_Component[i]=abs.(z-yDataTesting[1])
y_pred_top = zeros(200)
y_pred_bottom = zeros(200)
for i in 1:200
    y_pred_top[i] = mean(y_train[indices_top[i]])
    y_pred_bottom[i] = mean(y_train[indices_bottom[i]])
end

drawdown_top = zeros(200)
drawdown_bottom = zeros(200)
for i in 1:200
    drawdown_top[i] = mean(df_train[indices_top[i],:Drawdown])
    drawdown_bottom[i] = mean(df_train[indices_bottom[i],:Drawdown])
end
#first(X_train, 10)

[1693083, 2026018, 352985, 1949937, 1907759, 1360911, 1985741, 1922401, 443258, 605920, 565624, 1509372, 1922625, 353424, 1922538, 1334464, 1907195, 516023, 581672, 386075, 1971262, 207014, 937716, 538104, 1971314]


In [78]:
size(drawdown_top)

(200,)

In [81]:
using NearestNeighbors
function knn_prescription(y_pred_top, y_pred_bottom, drawdown_top, drawdown_bottom, C, D)
    Random.seed!(42)
    m = Model(Gurobi.Optimizer)
    set_optimizer_attribute(m, "OutputFlag", 0) 
    
    #we select the top 200 stocks to buy and the bottom 200 stocks to shorts
    @variable(m, w[1:400]>=0);
    @objective(m, Max, sum(w[i] * y_pred_top[i] for i in 1:200) + sum(w[i] * y_pred_bottom[i] for i in 1:200))
    
    #add the constraint that the max of drawdowns on the day 2022/06/23 weighted by w is less that C
    @constraint(m, (sum(w[i] * drawdown_top[i] for i in 1:200) + sum(w[i] * drawdown_bottom[i] for i in 1:200))/600 <= C)
    
    #sum of w should be equal to D
    @constraint(m, sum(w) == D)
    optimize!(m)
    #return the value of w
    return value.(w)

end

knn_prescription (generic function with 1 method)

In [84]:
w_opt_knn = knn_prescription(y_pred_top, y_pred_bottom, drawdown_top, drawdown_bottom, 0.01, 600)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18


400-element Vector{Float64}:
  0.0
 24.271722036636312
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  ⋮
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [85]:
sum(w_opt_knn)

600.0

In [86]:
sum(w_opt_knn[i] * df_2022_06_24_top[!,"Target"][i] for i in 1:200) + sum(w_opt_knn[i] * df_2022_06_24_bottom[!,"Target"][i] for i in 1:200)

1.918010727175761